In [1]:
from langchain.chat_models import ChatOpenAI

In [2]:
llm = ChatOpenAI(model_name="gpt-4")

In [3]:
import os
os.environ["LANGCHAIN_SESSION"] = "Docker"

In [4]:
f = """version: '3'
services:
  langchain-frontend:
    image: langchain/langchainplus-frontend:latest
    ports:
      - 80:80
    environment:
      - BACKEND_URL=http://langchain-backend:8000
      - PUBLIC_BASE_URL=http://localhost:8000
      - PUBLIC_DEV_MODE=true
      - LOG_LEVEL=WARNING
    depends_on:
      - langchain-backend
    build:
      context: frontend-react/.
      dockerfile: Dockerfile
    logging:
      driver: "json-file"
      options:
        max-size: "10m"
        max-file: "3"
        mode: "non-blocking"
        compress: "true"
        tag: "{{.Name}}"
        labels: "{{.Name}}"
        env: "TAG"

  langchain-backend:
    image: langchain/langchainplus-backend:latest
    environment:
      - PORT=8000
      - LANGCHAIN_ENV=local
      - LOG_LEVEL=WARNING
    ports:
      - 8000:8000
    depends_on:
      - langchain-db
    build:
      context: backend/.
      dockerfile: Dockerfile
    logging:
      driver: "json-file"
      options:
        max-size: "10m"
        max-file: "3"
        mode: "non-blocking"
        compress: "true"
        tag: "{{.Name}}"
        labels: "{{.Name}}"
        env: "TAG"

  langchain-db:
    image: postgres:14.1
    environment:
      - POSTGRES_PASSWORD=postgres
      - POSTGRES_USER=postgres
      - POSTGRES_DB=postgres
      - LOG_LEVEL=WARNING
    ports:
      - 5432:5432
    logging:
      driver: "json-file"
      options:
        max-size: "10m"
        max-file: "3"
        mode: "non-blocking"
        compress: "true"
        tag: "{{.Name}}"
        labels: "{{.Name}}"
        env: "TAG"
"""

In [5]:
prompt = ("I have a docker compose file:\n```\n{file}\n``` Could you update it"
" to restrict the output so all of the endpoints don’t get outputted to terminal? "
          "It's printing a lot of logs and I only want WARNING or ERROR logs to be visible"
          "Basically only want relevant warnings/errors ").format(file=f)

In [6]:
from langchain.agents import AgentType, initialize_agent

In [9]:
agent = initialize_agent([], llm=llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION)

In [11]:
res = agent({"input": prompt, "chat_history": []})

OutputParserException: Could not parse LLM output: `To restrict the output of the logs, you can update the logging configuration for each service in your Docker Compose file. You've already set the `LOG_LEVEL` environment variable to `WARNING` for all the services, which is good. However, you need to update the logging driver configuration to filter the logs based on the log level. 

Docker doesn't provide an out-of-the-box solution for filtering logs based on log level. But you can use a third-party logging driver like `fluentd` to achieve this. First, you need to install and configure Fluentd with the proper plugins.

Here's an example of how to modify your Docker Compose file to use Fluentd for logging:

1. Add a new `fluentd` service to your Docker Compose file:

```yaml
services:
  ...
  fluentd:
    image: fluent/fluentd:latest
    volumes:
      - ./fluentd/conf:/fluentd/etc
    ports:
      - "24224:24224"
      - "24224:24224/udp"
```

2. Create a `fluentd` directory and a `co

In [ ]:
print(res["output"])